In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os

# with open('C:/Users/weiso131/Desktop/paia2.4.5/resources/app.asar.unpacked/games/arkanoid/ml/graph/graph' + str(1) + '.pickle', 'rb') as f:
#     data = pickle.load(f)
#     for i in range(10):
#         print(data[0][i].shape)
        
    


In [2]:
x_data = []
y_label = []

for i in range(10000):
    path = 'C:/Users/weiso131/Desktop/paia2.4.5/resources/app.asar.unpacked/games/arkanoid/ml/graph/graph' + str(i) + '.pickle'
    if (os.path.exists(path)):
        
        with open(path, 'rb') as f:
            data = pickle.load(f)
        graphs, label = data[0], data[1]
        for graph in graphs:
            x_data.append(np.array([graph]))
            y_label.append(float(label))
    else:
        break

x_data = np.array(x_data)
y_label = np.array(y_label)
print(x_data.shape)
print(y_label.shape)

(84932, 1, 40, 100)
(84932,)


In [3]:
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"



# 將NumPy陣列轉換為PyTorch張量
x_data_tensor = torch.Tensor(x_data)  # 使用torch.Tensor默認轉換為float32
y_label_tensor = torch.Tensor(y_label) 

# 創建TensorDataset
dataset = TensorDataset(x_data_tensor, y_label_tensor)

# 分割數據集為訓練集和驗證集，比如90%訓練，10%驗證
train_size = int(0.9 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

batch = 64

# 創建DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch, shuffle=True)

# 現在train_dataloader和valid_dataloader都已經準備好，可以在訓練和驗證過程中使用了


In [4]:
for x, y in train_dataloader:
    print(x.shape, y.shape)
    break

torch.Size([64, 1, 40, 100]) torch.Size([64])


In [5]:


# 設定裝置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定義CNN模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        with torch.no_grad():
            self._to_linear = self.conv_layer(torch.zeros(1, 1, 40, 100))
            self._to_linear = torch.flatten(self._to_linear)
        self.fc_layer = nn.Sequential(
            nn.Linear(len(self._to_linear), 1024),
            nn.Dropout(0.7),
            nn.ReLU(),
            nn.Linear(1024, 1)
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = torch.flatten(x, 1)
        x = self.fc_layer(x)
        return x
    
# 定義NN模型
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()

        self._to_linear = torch.flatten(torch.zeros((1, 40, 100)))

        self.fc_layer = nn.Sequential(
            nn.Linear(len(self._to_linear), 4096),
            nn.Dropout(0.7),
            nn.ReLU(),
            nn.Linear(4096, 1024),
            nn.Dropout(0.7),
            nn.ReLU(),
            nn.Linear(1024, 1)
        )

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc_layer(x)
        return x

In [6]:
model = CNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
model

CNN(
  (conv_layer): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layer): Sequential(
    (0): Linear(in_features=6144, out_features=1024, bias=True)
    (1): Dropout(p=0.7, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=1024, out_features=1, bias=True)
  )
)

In [7]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    train_losses = []
    val_losses = []
    min_val_loss = 1e9
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device, dtype=torch.float32), labels.to(device, dtype=torch.float32)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.view(-1, 1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        model.eval()
        running_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device, dtype=torch.float32), labels.to(device, dtype=torch.float32)
                outputs = model(inputs)
                loss = criterion(outputs, labels.view(-1, 1))
                running_loss += loss.item()
        val_loss = running_loss / len(val_loader)

        if (val_loss < min_val_loss):
            print("save model")
            torch.save(model.state_dict(), "checkpoint2.pth")
            min_val_loss = val_loss

        val_losses.append(val_loss)

        print(f'Epoch {epoch+1}, Train Loss: {train_loss}, Val Loss: {val_loss}')
    
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Train vs Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [8]:
train_model(model=model, train_loader=train_dataloader, val_loader=valid_dataloader, criterion=criterion, optimizer=optimizer, epochs=1000)

save model
Epoch 1, Train Loss: 80.90052227834279, Val Loss: 49.70296954391594
save model
Epoch 2, Train Loss: 42.96852563036037, Val Loss: 33.788745263465366
save model
Epoch 3, Train Loss: 31.511565274673526, Val Loss: 25.04547821848016
save model
Epoch 4, Train Loss: 24.801741693408918, Val Loss: 21.919937119448097
save model
Epoch 5, Train Loss: 20.22736592871375, Val Loss: 17.708360815406742
save model
Epoch 6, Train Loss: 17.06445491782791, Val Loss: 14.964367414775648
save model
Epoch 7, Train Loss: 15.007019851796297, Val Loss: 14.559090392033857
save model
Epoch 8, Train Loss: 13.303308461400754, Val Loss: 11.96988334512352
save model
Epoch 9, Train Loss: 11.980064343009534, Val Loss: 11.622738744979515
save model
Epoch 10, Train Loss: 11.116134885364996, Val Loss: 11.088347456509009
save model
Epoch 11, Train Loss: 10.23248222023872, Val Loss: 9.80539796226903
save model
Epoch 12, Train Loss: 9.6930688706402, Val Loss: 8.995603971911553
save model
Epoch 13, Train Loss: 9.1322

KeyboardInterrupt: 